In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
!pip install geotext

In [ ]:
c_df = pd.read_csv('../input/covid19-tweets/covid19_tweets.csv')
c_df.head(10)

In [ ]:
c_df.shape

In [ ]:
c_df.info()

In [ ]:
c_df.describe()

In [ ]:
100*c_df.isnull().sum()/len(c_df)

In [ ]:
## Dropping the NA values, as imputing values would not be right for this dataset.
c_df.dropna(inplace = True)

In [ ]:
## Na Check
100*c_df.isnull().sum()/len(c_df)

## Exploratory Data Analysis and Data Visualization

### Location vs Tweet counts

In [ ]:
## Top 10 user_location or places of tweets.

tweet_location = c_df['user_location'].value_counts()
tweet_loc = tweet_location[:10]
tweet_loc

In [ ]:
tw_loc = pd.DataFrame(tweet_loc)
tw_loc = tw_loc.reset_index()
tw_loc.columns = ['Location', 'counts']
tw_loc

In [ ]:
plt.figure(figsize = (15,10))
sns.barplot(x = tw_loc.Location,y = tw_loc.counts, data = tw_loc)

In [ ]:
tweet_location

In [ ]:
Loc_count=pd.DataFrame(c_df['user_location'].value_counts())
Loc_count.reset_index(inplace = True)
Loc_count.rename(columns={'index':'Location','user_location':'count'},inplace=True)
Loc_count.sort_values(by='count',inplace=True,ascending=False)
Loc_count

In [ ]:
from geotext import GeoText
location=Loc_count.loc[2]['Location']
print(GeoText(location).countries)

In [ ]:
#!pip install geotext

In [ ]:
loc_country=Loc_count.copy()

In [ ]:
loc_country['Location']=loc_country['Location'].apply(lambda x:x.replace(',',' '))
loc_country['Location']=loc_country['Location'].apply(lambda x:(GeoText(x).country_mentions))
loc_country

In [ ]:
loc_country.drop(loc_country[loc_country['Location']=='[]'].index,inplace=True)
loc_country['Location']=loc_country['Location'].apply(lambda x:(x.keys()))
loc_country['Location']=loc_country['Location'].apply(lambda x:list(x))
loc_country

In [ ]:
loc_country.drop(loc_country.index[loc_country.Location.map(len)==0],inplace=True)
loc_country

In [ ]:
loc_country['Location']=loc_country['Location'].apply(lambda x:str(x[0]))
loc_country

In [ ]:
agg_func={'count':'sum'}
loc_country=loc_country.groupby(['Location']).aggregate(agg_func)
loc_country

In [ ]:
loc_country.sort_values(by=['count'],ascending=False,inplace=True)
loc_country.reset_index(inplace=True)
loc_country

In [ ]:
loc_country20 = loc_country.head(20)
loc_country20
plt.figure(figsize = (15,10))
sns.barplot(x = 'Location', y = 'count' ,data = loc_country20)
plt.title('Top 20 countries with highest count of tweets', fontweight = 'bold')

**Top 5 Countries with highest count of tweets:  US - USA, IN- India, GB- Great Britain, CA- Canada, AU- Australia**

### Date vs Tweet Counts

In [ ]:
## Data Analysis of the Date Column

date_df = c_df.copy()
date_df['date'].value_counts()

In [ ]:
date_df['Dates'] = pd.to_datetime(date_df['date']).dt.date
date_df['Time'] = pd.to_datetime(date_df['date']).dt.time

In [ ]:
date_count = date_df['Dates'].value_counts()
date_count = date_df['Dates'].value_counts().reset_index()
date_count = pd.DataFrame(date_count)
date_count.columns = ['Date', 'counts']

date_count

In [ ]:
plt.figure(figsize = (20,10))
sns.lineplot(x =date_count.Date, y = date_count.counts, color = 'red', marker = 'o')
plt.title('Number of Tweet per day')

In [ ]:
c_df.head()

### Hashtags Analysis

In [ ]:
ht_count = c_df['hashtags'].value_counts()
ht_count

In [ ]:
def htg(x):
    x = str(x)
    x = x.replace('[', '')
    x = x.replace(']', '')
    x = x.split(',')
    return x

## The hashtags are not clear and also not unique words, so the code below transformed it into unique words.

ht_count = c_df['hashtags'].value_counts().reset_index()
ht_count = pd.DataFrame(ht_count)
ht_count.columns = ['Hashtags', 'counts']
ht_count['Hashtags'] = ht_count['Hashtags'].apply(lambda x:htg(x))
ht_count = ht_count.explode('Hashtags')
ht_count['Hashtags'] = ht_count['Hashtags'].str.lower()
ht_count['Hashtags'] = ht_count['Hashtags'].str.replace(" ","")
ht_count['Hashtags'] = ht_count['Hashtags'].str.replace("'","")
ht_count10 = ht_count['Hashtags'].value_counts().reset_index()
ht_count10 = ht_count10[0:10]
ht_count10.rename(columns = {'index':'HashTag', 'Hashtags':'Count'}, inplace = True)
ht_count10

In [ ]:
plt.figure(figsize = (20,10))
sns.barplot(x = ht_count10.HashTag, y = ht_count10.Count, data = ht_count10)
plt.title('Top 10 Hashtags on Twitter', fontweight = 'bold', fontsize='15')

**Top 5 Hashtags**
1. covid19
2. coronavirus
3. pandemic
4. covid
5. India

### Source Analysis

In [ ]:
c_df.head()

In [ ]:
c_df['source'].value_counts()

In [ ]:
t_source = c_df['source'].value_counts().reset_index()
t_source = pd.DataFrame(t_source)
t_source.columns = ['Source of Tweet', 'counts']
t_source = t_source[t_source['counts']>=300]
t_source

In [ ]:
plt.figure(figsize = (30,15))
sns.barplot(x = 'Source of Tweet', y = 'counts', data = t_source)
plt.title('Sources of Tweets with count more than 300', fontweight = 'bold')

**Top 5 sources of Tweets:**
0.	Twitter Web App	
1.	Twitter for Android	
2.	Twitter for iPhone	
3.	TweetDeck	
4.	Hootsuite Inc.	

### Retweet Analysis

In [ ]:
rts = c_df['is_retweet'].value_counts().reset_index()

rts = pd.DataFrame(rts)
rts.columns = ['RT: True or False', 'counts']
rts

### Account Creation Analysis

In [ ]:
##finding out the date of the first tweet in dataset
date_col = date_df['Dates']
date_col.reset_index()
date_col = pd.DataFrame(date_col)
date_col.sort_values(by='Dates',inplace=True)
date_col['Dates'].iloc[0:1]

In [ ]:

bef_date = date_df[date_df['user_created'] < '2020-07-24']
aft_date = date_df[date_df['user_created'] >= '2020-07-24']
len1 = bef_date.shape[0]
len2 = aft_date.shape[0]
bef_perc = len1/(len1+len2)
aft_perc = len2/(len1+len2)
data = [['Before 1st Tweet',bef_perc],['After 1st Tweet',aft_perc]]
acc_creation = pd.DataFrame(data, columns = ['Category', 'Percent'])
acc_creation


In [ ]:
c_df.head()

### Username vs Count Analysis(Tweet Count)

In [ ]:
username = c_df['user_name'].value_counts().reset_index()
username = pd.DataFrame(username)
username.columns = ['Twitter UserName', 'Number of Tweets']
username = username.head(15)
username

In [ ]:
plt.figure(figsize = (30,10))
sns.barplot(x = 'Twitter UserName', y = 'Number of Tweets', data = username)

**Top 10 username with maximum number of tweets:**
0.	covidnews.ch	
1.	GlobalPandemic.NET	
2.	Blood Donors India	
3.	Hindustan Times	
4.	IANS Tweets	
5.	OTV	
6.	COVID Scale	
7.	Sambad English
8.	ANI	
9.	ABS-CBN News.

### User verified Analysis

In [ ]:
c_df['user_verified'].value_counts()

In [ ]:
plt.figure(figsize = (10,10))
sns.countplot('user_verified', data = c_df)

### Number of Followers Analysis

In [ ]:
new = c_df[['user_name', 'user_followers']].copy()
new = new.drop_duplicates(subset = ['user_name'])

new = new.sort_values(by = ['user_followers'], ascending = False)
new15 = new.head(15)
new15

In [ ]:
plt.figure(figsize = (30,15))
sns.barplot(x = 'user_name', y = 'user_followers', data = new15)
plt.xlabel('User Name', fontweight = 'bold')
plt.ylabel('No of Follower', fontweight = 'bold')
plt.title('Top 15 Twitter usernames by follower count')

**Top 5 Twitter pages in terms of twitter followers:**
1. National Geographic
2. CGTN
3. NDTV
4. The Times of India
5. United Nations

### Twitter Account creation Year Analysis

In [ ]:
year_df = c_df.copy()
year_df['Year'] = pd.to_datetime(year_df['user_created']).dt.year
year_df['Year'].value_counts()

In [ ]:
plt.figure(figsize = (20,10))
sns.countplot(year_df['Year'], data = year_df)

**The years 2009, 2010, 2011, 2012 and 2020 are the top 5 years in terms of account created counts on Twitter**

### Extracting Hour from time of Tweet and Analysis

In [ ]:
hour1 = c_df.copy()
hour1['hour_of_day'] = pd.to_datetime(hour1['date']).dt.hour
hour1.head()

In [ ]:
hour2 = hour1['hour_of_day'].value_counts().reset_index()
hour2 = pd.DataFrame(hour2)
hour2.columns = ['Hour of Day', 'count of tweets per hour']
hour2 = hour2.sort_values(by = ['count of tweets per hour'], ascending = False)
hour2


In [ ]:
plt.figure(figsize = (15,8))
sns.barplot(x = 'Hour of Day',y = 'count of tweets per hour', data = hour2)
plt.title('Tweet count by Hour of the Day', fontweight = 'bold')

**Top 10 hours of day which observe high number of tweet counts:**
0.	7 : 7AM	
1.	6 : 6AM	
2.	8 : 8AM
3.	5 : 5AM	
4.	4 : 4AM
5.	16 :4PM	
6.	3 :3AM	
7.	9 :9AM	
8.	17 : 5PM	
9.	18 : 6PM	

## Wordcloud Analysis

In [ ]:
from wordcloud import WordCloud, STOPWORDS
def word_cloud_func(c_df):
    word_cloud = WordCloud(
    background_color = 'black',
    stopwords = set(STOPWORDS),
    max_words = 50,
    max_font_size = 40,
    
    ).generate(str(c_df))
    fig = plt.figure(
    figsize = (20, 20),
    facecolor = 'k',
    edgecolor = 'k')
    plt.axis('off')
    
    fig.subplots_adjust(top=2.3)
    plt.imshow(word_cloud)
    plt.show()

### Word Cloud for all tweets in the dataset

In [ ]:
word_cloud_func(c_df['text'])

### WordCloud for top 5 users

- covidnews.ch
- GlobalPandemic.NET
- Blood Donors India
- Hindustan Times
- IANS Tweets

### For covidnews.ch

In [ ]:
temp_df = c_df[c_df['user_name']=='covidnews.ch']
word_cloud_func(temp_df['text'])

###  For GlobalPandemic.NET

In [ ]:
temp_df = c_df[c_df['user_name']=='GlobalPandemic.NET']
word_cloud_func(temp_df['text'])

### For Blood Donors India

In [ ]:
temp_df = c_df[c_df['user_name']=='Blood Donors India']
word_cloud_func(temp_df['text'])

### For Hindustan Times

In [ ]:
temp_df = c_df[c_df['user_name']=='Hindustan Times']
word_cloud_func(temp_df['text'])

### For IANS Tweets

In [ ]:
temp_df = c_df[c_df['user_name']=='IANS Tweets']
word_cloud_func(temp_df['text'])

### Some extra Analysis

### For Shashi Tharoor

In [ ]:
temp_df = c_df[c_df['user_name']=='Shashi Tharoor']
word_cloud_func(temp_df['text'])

### For NDTV

In [ ]:
temp_df = c_df[c_df['user_name']=='NDTV']
word_cloud_func(temp_df['text'])

### For WHO

In [ ]:
temp_df = c_df[c_df['user_name']=='World Health Organization (WHO)']
word_cloud_func(temp_df['text'])

## For OTV

In [ ]:
temp_df = c_df[c_df['user_name']=='OTV']
word_cloud_func(temp_df['text'])

## Word Cloud Analysis for User description

In [ ]:
word_cloud_func(c_df['user_description'])

## K-Means Clustering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
kc_df = c_df.copy()
vec = TfidfVectorizer(stop_words="english")
vec.fit(kc_df['text'].values)
features = vec.transform(kc_df['text'].values)

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(features)

In [ ]:
temp = kmeans.predict(features)
kc_df['Cluster'] = temp
kc_df

In [ ]:
len(kc_df[kc_df['Cluster'] == 0])

In [ ]:
len(kc_df[kc_df['Cluster'] == 1])

- **Number of Samples for Cluster 0: 89503**
- **Number of Samples for Cluster 1: 9635**

### Word Cloud for Cluster 0

In [ ]:
word_cloud_func(kc_df[kc_df['Cluster'] == 0]['text'])

### Word Cloud for Cluster 1

In [ ]:
word_cloud_func(kc_df[kc_df['Cluster'] == 1]['text'])

### Let's try for no of Clusters = 4

In [ ]:
kmeans1 = KMeans(n_clusters=4, random_state=0)
kmeans1.fit(features)

In [ ]:
temp1 = kmeans1.predict(features)
kc_df['Cluster4'] = temp1
kc_df

### Word Cloud for Cluster 0 (4 Clusters)

In [ ]:
word_cloud_func(kc_df[kc_df['Cluster4'] == 0]['text'])

### Word Cloud for Cluster 1 (4 Clusters)

In [ ]:
word_cloud_func(kc_df[kc_df['Cluster4'] == 1]['text'])

### Word Cloud for Cluster 2 (4 Clusters)

In [ ]:
word_cloud_func(kc_df[kc_df['Cluster4'] == 2]['text'])

### Word Cloud for Cluster 3 (4 Clusters)

In [ ]:
word_cloud_func(kc_df[kc_df['Cluster4'] == 3]['text'])